### Fuzzy Classifier Testing

In [1]:
require 'open-uri'
require 'json'
require 'daru'
require 'distribution'
require 'sqlite3'
require 'rbplotly'


Install the spreadsheet gem version ~>1.1.1 for using spreadsheet functions.

Install the mechanize gem version ~>2.7.5 for using mechanize functions.


"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

true

In [2]:
dir = "#{ENV['HOME']}/cs6140/final_project"
db = SQLite3::Database.new "#{dir}/credit_risk_data.db"
db.results_as_hash = true

true

In [50]:
features = ["EXT_SOURCE_3"]
features1 = ["EXT_SOURCE_3", "EXT_SOURCE_2", "EXT_SOURCE_1", "DAYS_BIRTH"]

def fetch_data db, features
  table = "application_train"
  query = "select " + features.join(",") + ", Target label from " + table + " order by " + features[0] + " asc" 
  data = []
  db.execute(query).each do |r|
    row = Hash.new
    row["features"] = Hash.new
    features.each do |f|
      row["features"][f] = r[f]
    end
    row["label"] = r["label"]
    data << row
  end
  return data
end

data = fetch_data db, features
data1 = fetch_data db, features1
1

1

In [5]:
def fetch_data_special db
  table = "application_train"
  query = "select EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3, DAYS_BIRTH, (AMT_CREDIT/AMT_ANNUITY) as AMT_RATIO, Target label from " + table
  data = []
  db.execute(query).each do |r|
    row = Hash.new
    row["features"] = Hash.new
    ["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3", "AMT_RATIO", "DAYS_BIRTH"].each do |f|
      row["features"][f] = r[f]
    end
    row["label"] = r["label"]
    data << row
  end
  return data
end

# data1 = fetch_data_special db
1

1

In [6]:
data1 = fetch_data_special db
data1[0]

{"features"=>{"EXT_SOURCE_1"=>0.08303696739132256, "EXT_SOURCE_2"=>0.2629485927471776, "EXT_SOURCE_3"=>0.13937578009978951, "AMT_RATIO"=>16.46110402623429, "DAYS_BIRTH"=>-9461}, "label"=>1}

In [7]:
scores = data.collect {|x| x["features"]["EXT_SOURCE_3"]}.select{ |x| x && x!= "" }.sort.uniq.reverse
1

1

In [8]:
scores[0, 10]

[0.8960095494948396, 0.8939760746042866, 0.8876642018413868, 0.8854883941521002, 0.8825303127941461, 0.881026574983999, 0.8802684803872619, 0.8795062158892447, 0.878739766981187, 0.8771942581973663]

IVFS, Tuning parameters $\alpha, a, b, W$ where a and b are tuning parameters for IV-REF, IV-REF is a function to understand the similarities of two intervals

### Step 0. Preparation, helper functions and classes

In [9]:
# turns data to data_class
def parse_class x
  s = Hash.new {|h,k| h[k] = 0}
  x.each do |p|
    s[p["label"]] += 1
  end
  return s
end

:parse_class

In [51]:
def roc_curve train, predictions, scores
  tprs = []
  fprs = []    
        
  scores.each do |s|
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    train.each.with_index do |row, i|
      y = row["label"]
      p = (predictions[i].values[0] > s) ? 1 : 0
      if y == p
        tp += 1 if p.to_i.to_s == "1"
        tn += 1 if p.to_i.to_s == "0"
      else
        fp += 1 if p.to_i.to_s == "1"
        fn += 1 if p.to_i.to_s == "0"
      end
    end
    tpr = (tp + fn) == 0.0 ? 0.0 : tp.to_f / (tp + fn) 
    fpr = (fp + tn) == 0.0 ? 0.0 : fp.to_f / (fp + tn)
    tprs << tpr
    fprs << fpr
  end
  
  tprs << 1.0
  fprs << 1.0
  
  return fprs, tprs
end

def auc fprs, tprs
  x = 0.0
  y = 0.0
  area = 0.0
  fprs.each.with_index do |fpr, i|
    tpr = tprs[i]
    area += (fpr - x) * (tpr + y) / 2.0
    x = fpr
    y = tpr
  end
  return area
end

def plot x, y
  Daru::DataFrame.new({x: x, y: y}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
    plot.x_label "X"
    plot.y_label "Y"
  end
end

def confusion_matrix cls_names, x, predictions
  counts = Hash.new {|h,k| h[k] = Hash.new {|h,k| h[k] = 0}}
  cls_names.each {|i| cls_names.each{|j| counts[i][j] = 0}}
  
  x.each.with_index do |row, i|
    predicted_label = predictions[i].keys.first
    actual_label = row["label"]
    
    counts[predicted_label.to_s][actual_label.to_s] += 1
  end
  
  return counts
end

def update_weights(w, dw, learning_rate)
  w1 = w.clone
  dw.each_key do |k|
    w1[k] -= learning_rate * dw[k]
  end  
  w1
end

:update_weights

## Step 1. Generate type-1 FRBCS and rule base

### 1.1 Calculate the  minimum support for each class

In [52]:
parse_class data

{0=>282686, 1=>24825}

In [53]:
# return the minimum support in order to generate a rule
def min_supp data, label, minsups
  x_class = parse_class data
  return minsups[label] * x_class[label].to_f / x_class.values.sum
end

:min_supp

$MinSupp_{C_{j}} = minSup * f_{C_{j}}$, where $f_{C_{j}}$ is the proportion of class k to the entire data

We will use 0.1 and 0.2 as minSup for each class respectively.

In [54]:
# calculate the minimum support
minsups = [0.1, 0.12]
min_supps = Hash.new
min_supps[0] = min_supp data, 0, minsups
min_supps[1] = min_supp data, 1, minsups
puts min_supps

{0=>0.09192711805431351, 1=>0.009687458334823795}


### 1.2 Build the search tree 
We will use the following 5-label linguistic fuzzy sets to train the numerical patterns for search tree[1] with the triangle membership function.

Each Sets stands for:

$S_{0}$: very low, $S_{1}$: low, $S_{2}$: medium, $S_{3}$: high, $S_{4}$: very high, 

In [55]:
# Generate the n-label linguistic fuzzy sets based on all possible features
def feature_fuzzy_sets data, n
  fs = []
  h = data[0]["features"].keys
  h.each do |f|
    scores = data.collect {|x| x["features"][f]}.select{ |x| x && x!= "" }.
      sort.uniq
    # 5-label fuzzy set
    (0..(n-1)).each do |i|
      max = scores.max
      min = scores.min
      # since min is 0 and max is 1, there will be only 4 gaps
      base = (max - min).to_f / (n-1)
      r = Hash.new
      r["feature"] = f
      r["set"] = i
      r["mid"] = min + base * i
      r["base"] = base * 2
      fs << r
    end
  end
  return fs
end

fs = feature_fuzzy_sets data, 5

[{"feature"=>"EXT_SOURCE_3", "set"=>0, "mid"=>0.0005272652387098817, "base"=>0.4477411421280649}, {"feature"=>"EXT_SOURCE_3", "set"=>1, "mid"=>0.22439783630274232, "base"=>0.4477411421280649}, {"feature"=>"EXT_SOURCE_3", "set"=>2, "mid"=>0.44826840736677476, "base"=>0.4477411421280649}, {"feature"=>"EXT_SOURCE_3", "set"=>3, "mid"=>0.6721389784308072, "base"=>0.4477411421280649}, {"feature"=>"EXT_SOURCE_3", "set"=>4, "mid"=>0.8960095494948396, "base"=>0.4477411421280649}]

### Fuzzy seach Tree

Then we define the $Support$ and $Confidence$ functions, these will be the pruning criteras for the search tree

$\textbf Support$ ($A_{j} \rightarrow C_{j}) = \frac {\sum x_{p \in Class C_{j} } \mu_{A_{j}}(x_{p}) }{|N|}$

$\textbf Confidence(A_{j} \rightarrow C_{j}) = \frac {\sum x_{p \in Class C_{j} } \mu_{A_{j}}(x_{p}) }{\sum_{N} \mu_{A_{j}}(x_{p})}$

In [15]:
# def density data, ante, w
#   u = 0.0
#   i = 0.0
#   data.each do |x|
#     v = x["features"][ante["feature"]]
#     next if v.nil? or v == ""
#     i += 1
#     u += (1.0 - (mu x, ante, w, true))**2
#   end
#   return u/i
# end

def support data, antes, cl, mu_table
  u = 0.0
  data.each do |x|
    next if x["label"] != cl
    mus = []
    antes.each do |ante|
      v = x["features"][ante["feature"]]
      next if v.nil? or v == ""
      header = ante["feature"].to_s + "_" + ante["set"].to_s
      mu = mu_table[header][v]
      mus << mu
      # skip some iterations if we found one mu as 0.
      break if mu == 0
    end
    u += t_norm mus if mus.length > 0
  end
  return u/data.length
end

def confidence data, antes, cl, mu_table
  u = 0.0
  d = 0.0
  data.each do |x|
    mus = []
    antes.each do |ante|
      v = x["features"][ante["feature"]]
      next if v.nil? or v == ""
      header = ante["feature"].to_s + "_" + ante["set"].to_s
      mu = mu_table[header][v]
      mus << mu
      # skip some iterations if we found one mu as 0.
      break if mu == 0
    end
    u += t_norm mus if x["label"] == cl && mus.length > 0
    d += t_norm mus if mus.length > 0
  end
  return u/d
end

# def support_confidence data, antes, cl, mu_table
#   u = 0.0
#   d = 0.0
#   i = 0
#   data.each do |x|
#     mus = []
#     antes.each do |ante|
#       v = x["features"][ante["feature"]]
#       next if v.nil? or v == ""
#       i += 1
#       header = ante["feature"].to_s + "_" + ante["set"].to_s
#       mu = mu_table[header][v]
#       mus << mu
#       # skip some iterations if we found one mu as 0.
#       break if mu == 0
#     end
#     u += t_norm mus if x["label"] == cl && mus.length > 0
#     d += t_norm mus if mus.length > 0
#   end
#   return [u/data.length, u/d]
# end

# Given a fuzzy set, calculate its target mu
def mu x, ante, w, negative_enabled = false
  f = ante["feature"]
  mid = ante["mid"]
  base = ante["base"]
  v = x["features"][f]
  puts "gaga" if v == "" or v.nil?
  mem = 1.0 - ((v - (mid + w * ante["base"])).to_f * 2 / base).abs
  return mem if negative_enabled
  return [0.0, mem].max
end

# Calulate the t-normalization of a vecotr x, use either min or product t-norm
def t_norm x
  return x.min
end

def t_norm x
  return x.reduce(:*)
end

:t_norm

In [16]:
def supp_prune rules, min_supp
  return rules.select { |r| r["support"] > min_supp}
end

def confi_prune rules
  return rules.select { |r| r["confidence"] > 0.0 and r["confidence"] < 1.0 }
end

def generate_leaves data, last_leaves, mu_table, cl
  leaves = []
  last_leaves.each do |rule1|
    f1 = rule1["antecedent"].map { |r| r["feature"] }
    last_leaves.each do |rule2|
      f2 = rule2["antecedent"].map { |r| r["feature"] }
      total = (f1 + f2).uniq.length
      next if total < f1.length + f2.length
      rule = Hash.new
      rule["antecedent"] = (rule1["antecedent"] + rule2["antecedent"])
      rule["support"] = support data, rule["antecedent"], cl, mu_table
      rule["confidence"] = confidence data, rule["antecedent"], cl, mu_table
      rule["class"] = cl
      leaves << rule
    end
  end
  return leaves
end

def generate_mu_table data, ffs, w
  mu_table = Hash.new

  # Initialize the rules
  ffs.each do |fs|
    (0..1).each do |cl|
      header = fs["feature"].to_s + "_" + fs["set"].to_s
      mu_table[header] = Hash.new
      data.each do |x|
        v = x["features"][fs["feature"]]
        mu_table[header][v] = (mu x, fs, w[header]) if !mu_table[header].key?(v) and v != "" and !v.nil?
      end
    end
  end
  return mu_table
end

:generate_mu_table

In [17]:
# def adjust_data data, ante
#   data_ad = []
#   data.each do |x|
#     v = x["features"][ante["feature"]]
#     next if v.nil? or v == ""
#     mid = ante["mid"]
#     range = ante["base"] * 0.75
#     data_ad << x if (v - mid).abs <= range
#   end
#   return data_ad
# end

# def adjust w
#     w = [[-0.25, w].max, 0.25].min
# end

# def grad data, ante, w
#   u = 0.0
#   i = 0
#   data.each do |x|
#     v = x["features"][ante["feature"]]
#     next if v.nil? or v == ""
#     i += 1
#     u += - 4 * ( v - (ante["mid"] + w * ante["base"]) ) / (ante["base"] ** 2)
#   end
#   return u / i
# end

In [18]:
# w = Hash.new { |h,k| h[k] = 0.0 }
# ffs = feature_fuzzy_sets data, 5

# density data, ffs[0], 0

In [19]:
# w = Hash.new { |h,k| h[k] = 0.0 }

# # learning rate and tolerence as usual
# lr = 1e-2
# tol = 1e-3
# max_iter = 100

# ffs.each do |fs|
#   # Adjust the data to only include relavent features among relavent region from [-0.75, 0.75]
#   data_ad = adjust_data data, fs
#   header = fs["feature"].to_s + "_" + fs["set"].to_s
#   wi = 0.0
#   prev_wi = 0.0
#   (0..max_iter).each do
#     dw = grad data_ad, fs, wi
#     wi = adjust(wi - lr*dw)
# #     puts density data_ad, fs, wi
# #     puts wi
#     if (prev_wi - wi).abs < tol
#       w[header] = wi
#       break
#     end
#     prev_wi = wi
#     w[header] = wi
#   end
# end
# 1

# puts w

In [56]:
# Build a search tree based on five-label fuzzy set
def build_search_tree data, min_supps, max_lvl, n, w
  # Generate the first layer of rules, getting all possible fuzzy sets by the 5-label linguistic rule
  last_leaves = Hash.new { |h, k| h[k] = []}
  ffs = feature_fuzzy_sets data, n
  
  puts "Initialize mu table"
  
  mu_table = generate_mu_table data, ffs, w
  
  puts "Mu table initialization complete"
  puts "Generate first lvl of leaves"
  
  ffs.each do |fs|
    (0..1).each do |cl|
      rule = Hash.new
      rule["antecedent"] = [fs]
#       rule["support"], rule["confidence"], = support_confidence data, rule["antecedent"], cl, mu_table
      rule["support"] = support data, rule["antecedent"], cl, mu_table
      rule["confidence"] = confidence data, rule["antecedent"], cl, mu_table
      rule["class"] = cl
      last_leaves[cl] << rule
    end
  end
  
  rules = Hash.new { |h, k| h[k] = []}
 
  (0..1).each do |cl|
    curr_lvl = 0
    print "Genearte search tree for label ", cl, "\n"
    while true
      curr_lvl += 1
      print "\t lvl ", curr_lvl, "\n"
      last_leaves[cl] = (supp_prune last_leaves[cl], min_supps[cl])
      rules[cl] += last_leaves[cl]
      break if curr_lvl >= max_lvl || last_leaves[cl].length <= 2
      last_leaves[cl] = (confi_prune last_leaves[cl])
      last_leaves[cl] = generate_leaves data, last_leaves[cl], mu_table, cl
    end
  end
  
  return rules
end

:build_search_tree

In [57]:
w0 = Hash.new { |h,k| h[k] = 0.0 }
w = w0
rb = []
# max lvl of the tree
max_lvl = 3

# print the parameters
puts min_supps, max_lvl

# define the rule base

rb = build_search_tree data[0, 10000], min_supps, max_lvl, 5, w
1

{0=>0.09192711805431351, 1=>0.009687458334823795}
3
Initialize mu table
Mu table initialization complete
Generate first lvl of leaves
Genearte search tree for label 0
	 lvl 1
	 lvl 2
Genearte search tree for label 1
	 lvl 1
	 lvl 2


1

In [58]:
puts rb[0].length
puts rb[1].length
puts rb[0][0, 3]

3
5
[{"antecedent"=>[{"feature"=>"EXT_SOURCE_3", "set"=>2, "mid"=>0.0695200711486511, "base"=>0.06899280590994121}], "support"=>0.17587045895347858, "confidence"=>0.7307487398582146, "class"=>0}, {"antecedent"=>[{"feature"=>"EXT_SOURCE_3", "set"=>3, "mid"=>0.10401647410362169, "base"=>0.06899280590994121}], "support"=>0.2754752559852643, "confidence"=>0.773186552904449, "class"=>0}, {"antecedent"=>[{"feature"=>"EXT_SOURCE_3", "set"=>4, "mid"=>0.1385128770585923, "base"=>0.06899280590994121}], "support"=>0.17534415170186235, "confidence"=>0.7894406692721149, "class"=>0}]


In [59]:
w0 = Hash.new { |h,k| h[k] = 0.0 }
w = w0
rb = []
# max lvl of the tree
max_lvl = 3

# print the parameters
puts min_supps, max_lvl

rb1 = []
rb1 = build_search_tree data1.shuffle[0, 30000], min_supps, 5, 5, w
1

{0=>0.09192711805431351, 1=>0.009687458334823795}
3
Initialize mu table
Mu table initialization complete
Generate first lvl of leaves
Genearte search tree for label 0
	 lvl 1
	 lvl 2
	 lvl 3
Genearte search tree for label 1
	 lvl 1
	 lvl 2
	 lvl 3


1

In [24]:
puts rb1[0].length
puts rb1[1].length
puts rb1[0].length + rb1[1].length
puts rb1[0][0, 3]

136
94
230
[{"antecedent"=>[{"feature"=>"EXT_SOURCE_1", "set"=>1, "mid"=>0.24815057330223564, "base"=>0.45645030012747206}], "support"=>0.0980856332157271, "confidence"=>0.8934817348861779, "class"=>0}, {"antecedent"=>[{"feature"=>"EXT_SOURCE_1", "set"=>2, "mid"=>0.47637572336597167, "base"=>0.45645030012747206}], "support"=>0.1367529092354696, "confidence"=>0.9296142868608361, "class"=>0}, {"antecedent"=>[{"feature"=>"EXT_SOURCE_1", "set"=>3, "mid"=>0.7046008734297078, "base"=>0.45645030012747206}], "support"=>0.12696833804064317, "confidence"=>0.9583391352626806, "class"=>0}]


In [25]:
# switch between different data
rbs = rb1[0] + rb1[1]
data = data1
1
puts data[0]

{"features"=>{"EXT_SOURCE_1"=>0.08303696739132256, "EXT_SOURCE_2"=>0.2629485927471776, "EXT_SOURCE_3"=>0.13937578009978951, "AMT_RATIO"=>16.46110402623429, "DAYS_BIRTH"=>-9461}, "label"=>1}


In [26]:
# initialize the mu table, which will be used heavily later on
ffs = feature_fuzzy_sets data, 5
  
mu_table = generate_mu_table data, ffs, w
puts "Mu table initialization complete"

Mu table initialization complete


### Step 2. Candidate rules prescreening(deprecated)

In this step, the generated rule base will be prescreened based on a quality measurement value wwr_acc. 
  
There will be two rounds, first, create a weight for each row and initialized it to 1. In round 1, the wwr_acc of all rules will be calculated, then the weight of rows covered the ruless will be updated one by one based on the ranking of wwr_acc.

In [27]:
# def wwr_acc rule, data, w, mu_table  
#   cl = rule["class"]
#   n = 0.0
#   n_cj = 0.0
#   n_prime_cj = 0.0
#   n_double_prime_a = 0.0
#   n_double_prime_acj = 0.0
  
#   data.each.with_index do |x, i|    
#     mus = []   
#     rule["antecedent"].each do |ante|
#       v = x["features"][ante["feature"]]
#       next if v.nil? or v == ""
#       header = ante["feature"].to_s + "_" + ante["set"].to_s
#       mus << mu_table[header][v]
#     end
    
#     mu = mus.length > 0 ? (t_norm mus) : 0.0
    
#     n += 1
#     n_double_prime_a += w[i]*mu
    
#     if x["label"] == cl
#       n_cj += 1
#       n_prime_cj += w[i]
#       n_double_prime_acj += w[i]*mu
#     end
#   end
#   # print n_cj, "   ", n_prime_cj, "   ", n_double_prime_acj, "   ", n_double_prime_a, "\n"
#   return (n_double_prime_acj / n_prime_cj) * (n_double_prime_acj / n_double_prime_a - n_cj / n)
# end

In [28]:
# def update_weight w, dw, rule, data, mu_table
#   data.each.with_index do |x, i|    
#     mus = []
#     cl = rule["class"]
#     rule["antecedent"].each do |ante|
#       v = x["features"][ante["feature"]]
#       next if v.nil? or v == ""
#       header = ante["feature"].to_s + "_" + ante["set"].to_s
#       mus << mu_table[header][v]
#     end
    
#     mu = mus.length > 0 ? (t_norm mus) : 0.0
#     w[i] = (mu != 0.0 and x["label"] == cl) ? w[i] + dw : w[i]
#   end
# end

In [29]:
# w = Hash.new { |h, k| h[k] = 1.0 }

# puts "start rbs ranking " + j.to_s
# rbs.each do |r|
#   r["wwr_acc"] = wwr_acc r, data, w, mu_table
# end

# rbs_sorted = rbs.sort_by{ |r| -r["wwr_acc"] }

# puts "Then update weight"

# rbs_sorted.each.with_index do |rb, i|
#   dw = -1.0 / (i+2)
#   update_weight w, dw, rb, data, mu_table
#   z = w.values.sum
#   w.each{ |k,v| w[k] = v.to_f / z }
# end
# 1

In [30]:
# puts "rank again " + j.to_s
# rbs.each do |r|
#   r["wwr_acc"] = wwr_acc r, data, w, mu_table
# end
# 1

In [31]:
# rbs_prescreened = rbs.sort_by{ |r| -r["wwr_acc"] }.select{ |r| r["wwr_acc"] > 0.0 }
# puts rbs.length
# puts rbs_prescreened.length

In [32]:
# rbs_prescreened[0, 10]

### Step 3. IVFSs construction

In this step, we will construct the interval-valued fuzzy set based on our original type-1 fuzzy set.

![IVFS](IVFS_sample.png)

The lower bound of IVFS is the rule we already have from Step 1. The upper bound of IVFS will be the one shown in the graph. However, instead of altering or construct a new rbs structure, we will make a new mu_table to handle the interval problem

In [33]:
def mu_iv x, ante, w
  f = ante["feature"]
  mid = ante["mid"]
  base = ante["base"]
  v = x["features"][f]
  puts "gaga" if v == ""  
  mem = 1.0 - ((v - (mid + w * ante["base"])).to_f * 2 / base).abs
  mem_up = 1.0 - ((v - (mid + w * ante["base"])).to_f * 2 / (base*1.5)).abs
  return [0.0, mem].max, [0.0, mem_up].max
end

def generate_mu_iv_table data, ffs, w
  mu_table = Hash.new
  # Initialize the rules
  ffs.each do |fs|
    (0..1).each do |cl|
      header = fs["feature"].to_s + "_" + fs["set"].to_s
      mu_table[header] = Hash.new
      data.each do |x|
        v = x["features"][fs["feature"]]
        if !mu_table[header].key?(v) and v != "" and !v.nil?
          mu_table[header][v] = Hash.new         
          mu_table[header][v]["lo"], mu_table[header][v]["hi"] = (mu_iv x, fs, w[header])
        end
      end
    end
  end
  return mu_table
end

:generate_mu_iv_table

In [34]:
ffs = feature_fuzzy_sets data, 5
  
puts "Initialize mu table"
mu_iv_table = generate_mu_iv_table data, ffs, w0
puts "Mu table initialization complete"

Initialize mu table
Mu table initialization complete


### Step 4. $\alpha$ Tuning(deprecated)

![lfs](linguistic-fuzzy-sets.png)

### Step 5. Evaluate IV-REFs
IV-REFs is the interval-valued restricted function, it tells the similarity between two intervals, when we use this to compare with the ideal interval [1, 1], it gives the error of system which can be used to tune the weights, IV-REFs also has two paramters for tuning, which are a and b.

In [35]:
def iv_refs iv1, iv2
end

:iv_refs

### Step 10.  IV-FRM, Calculate the Vote (example is covered in $RB$)

$V_{class_{k}} (x_{p}) = f_{R_{j} \in RB \space and \space C_{j} = k}(\mu_{A_{j}}(x_{p}) * RW_{j})$

Explanation:

$V_{class_{k}} (x_{p})$ gives the score for $x_{p}$ given the rule set $RB$ and $class_k$.

$\mu_{A_{j}}(x_{p})$ gives the memembership/reliability of applying $R_{j}$ to the data instance $x_{p}$, $RW_{j}$ defines the weight of $R_{j}$

$f$ is the aggregation function, which can be either maximum or summation.

In [36]:
def predict x, rb, mu_table
  votes = Hash.new { |h,k| h[k] = 0.0 }
  (0..1).each do |cl|
    rb.each do |r|
      next if r["class"] != cl
      r["antecedent"].each do |ante|
        v = x["features"][ante["feature"]]
        next if v.nil? or v == ""
        header = ante["feature"].to_s + "_" + ante["set"].to_s
        votes[cl] += r["support"] * r["confidence"] * mu_table[header][v]
      end
    end
  end
  return votes
end

def predict_iv x, rb, mu_table
  votes = Hash.new { |h,k| h[k] = 0.0 }
  (0..1).each do |cl|
    rb.each do |r|
      next if r["class"] != cl
      r["antecedent"].each do |ante|
        v = x["features"][ante["feature"]]
        next if v.nil? or v == ""
        header = ante["feature"].to_s + "_" + ante["set"].to_s
        votes[cl] += r["support"] * r["confidence"] * 
        ( 1 - ( mu_table[header][v]["hi"] - mu_table[header][v]["lo"] ) ) *  mu_table[header][v]["hi"]
      end
    end
  end
  return votes
end

:predict_iv

In [37]:
rbs_target = rbs
1

1

In [38]:
x = data[0]
p = predict x, rbs_target, mu_table

{0=>6.5614393522244, 1=>0.07618830961558865}

In [39]:
# now it's the time to do actual predictions
predictions = []
l = data.length/2
train = data.shuffle[0, l]
# l = 20

train.each.with_index do |x, i|
  total = 0.0
#   s = predict x, rbs_target, mu_table
  s = predict x, rbs_target, mu_table
  s[0] = s[0] * 0.015
  max = [s[0], s[1]].max
  predictions << { s.key(max) => s[0] == 0.0 ? 100.0 : s[1] / s[0] }
end

m = confusion_matrix(%w(0 1), train, predictions)
puts m
scores = predictions.collect{ |k| k.values[0] }.uniq.sort.reverse
puts scores[0, 10]
1

{"0"=>{"0"=>141379, "1"=>12369}, "1"=>{"0"=>7, "1"=>0}}
[1.1686140827535838, 1.0838295543152674, 1.0738268527781354, 1.0285270459817608, 1.0271900287854425, 1.0165451838134416, 1.013275638805112, 0.9954964456980429, 0.9856748176530624, 0.982366577132025]


1

In [40]:
puts scores.length
scores_limited = scores.select.with_index {|x, i| i % 1000 == 0}
puts scores_limited.length

153755
154


In [41]:
scores_limited[15]

0.5716803293154914

In [42]:
fprs, tprs = roc_curve train, predictions, scores_limited
puts fprs[0, 10]
puts tprs[0, 10]
plot(fprs, tprs)

[0.0, 0.005276335705091027, 0.01039706901673433, 0.01586437129560211, 0.02128216372200925, 0.026742393164811227, 0.03226627813220545, 0.03787503713238934, 0.043363557919454544, 0.04903597244423069]
[0.0, 0.02053520899021748, 0.04284905812919395, 0.06120139057320721, 0.08011965397364379, 0.0985528336971461, 0.116258387905247, 0.13299377475947935, 0.15110356536502548, 0.16711132670385642]


#<Nyaplot::Plot:0x000000003612f850 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000000035d47738 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"3a0923ce-3798-4fbb-9a29-d5fc20c48404"}, @xrange=[0.0, 1.0], @yrange=[0.0, 1.0]>], :options=>{:x_label=>"X", :y_label=>"Y", :zoom=>true, :width=>700, :xrange=>[0.0, 1.0], :yrange=>[0.0, 1.0]}}>

In [43]:
auc fprs, tprs 

0.6917283732300815

## IV prediciton

In [44]:
x = data[0]
p = predict_iv x, rbs_target, mu_iv_table

{0=>8.133535186997033, 1=>0.08810925372020774}

In [45]:
# now it's the time to do actual predictions
predictions = []
l = data.length/2
# l = 20

train.each.with_index do |x, i|
  total = 0.0
  s = predict_iv x, rbs_target, mu_iv_table
  max = [s[0], s[1]].max
  predictions << { s.key(max) => s[0] == 0.0 ? 100.0 : s[1] / s[0] }
end

m = confusion_matrix(%w(0 1), train, predictions)
puts m
scores = predictions.collect{ |k| k.values[0] }.uniq.sort.reverse
puts scores[0, 10]
1

{"0"=>{"0"=>141386, "1"=>12369}, "1"=>{"0"=>0, "1"=>0}}
[0.012994695384979781, 0.012828565170477547, 0.01267131744435256, 0.012594073987328553, 0.012570252546990623, 0.012503670949391622, 0.012456034744269684, 0.012429789325083714, 0.012399999916028463, 0.012381186733746266]


1

In [46]:
puts scores.length
scores_limited = scores.select.with_index {|x, i| i % 1000 == 0}
puts scores_limited.length

153755
154


In [47]:
fprs, tprs = roc_curve train, predictions, scores_limited
puts fprs[0, 10]
puts tprs[0, 10]
plot(fprs, tprs)

[0.0, 0.005170243164103943, 0.010262685131484022, 0.015432928295587964, 0.020900230574455746, 0.02652313524677125, 0.03191263632891517, 0.037464812640572616, 0.0429391877555062, 0.04850550973929526]
[0.0, 0.021747918182553158, 0.04438515643948581, 0.06613307462203896, 0.08448540706605223, 0.1010590993613065, 0.12030075187969924, 0.13768291696984397, 0.15595440213436818, 0.1731748726655348]


#<Nyaplot::Plot:0x00000000393206e8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000000038ef00c8 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"97b9d878-d425-4877-b8ab-437e67b16a9e"}, @xrange=[0.0, 1.0], @yrange=[0.0, 1.0]>], :options=>{:x_label=>"X", :y_label=>"Y", :zoom=>true, :width=>700, :xrange=>[0.0, 1.0], :yrange=>[0.0, 1.0]}}>

In [48]:
auc fprs, tprs 

0.697386025947156

### Things need to be understood in list

1. Type-1 Fuzzy Set - done
2. Interval-Valued Fuzzy Set - 
3. Association Discovery classifier
4. IV-REFs
5. How to determine if a example is covered or not
6. how to handle uncovered examples

### Proposed Steps

1. Type-1 Fuzzy Set Generations
2. Rule Base Generation
3. IVFSs Construction
5. IVFRM - Interval-valued fuzzy reasoning method, basically predict the label
6. tuning systm parameters

### Explanation:
1. IVFRM - Interval-valued fuzzy reasoning method, basically predict the labe

ok2